In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.signal import savgol_filter
from shabanipy.jj.utils import (correct_voltage_offset,
                                compute_resistance)
from shabanipy.jj.shapiro.utils import correct_voltage_offset_per_power
from shabanipy.jj.shapiro.binning import bin_power_shapiro_steps,extract_step_weight
from shabanipy.jj.shapiro.plotting_shapiro import (plot_differential_resistance_map,
                                           plot_shapiro_histogram,
                                           plot_step_weights)

#: Name of the sample that must appear in the measurement name usually of the form "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"
SAMPLE_NAME = "{Wafer}-{Piece}_{Design}-{Iteration}"
SAMPLE_ID = "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"

#: hdf5 file number
FILE_NUM = ''

#: Path to store generated files
PATH = (f"/Users/bh/Desktop/Code/Topological JJ/Samples/{SAMPLE_NAME}/{SAMPLE_ID}")

#: Name of generated processed data file
PROCESSED_DATA_NAME = (f"{PATH}/Data/{SAMPLE_ID}_processed-data-{FILE_NUM}.hdf5")

h = h5py.File(PROCESSED_DATA_NAME, 'r')

field_y = 'In-plane Field - Y
field_z = 'In-plane Field - Z'
freq = 'Microwave: Frequency::'
freq_val = float(freq[22:])
vg = 'Vg::'

f = h['Data'][f'{freq}'][f'{field_y}'][f'{field_z}'][f'{vg}']

In [ ]:
data = np.array(f["Voltage Drop"])
scaled_v_drop = np.array(f["ScaledVoltage"])
bias = np.array(f["Bias"])
dVdI = np.diff(np.array(f["ScaledVoltage"]))/np.diff(np.array(f["Bias"]))
dR = np.array(f["dR"])
power = np.array(f["Microwave - Power"])

In [ ]:
coarse_corrected_data = correct_voltage_offset(bias,data,n_peak_width = 1)
fine_corrected_data = correct_voltage_offset_per_power(power,bias,
                                                       coarse_corrected_data,frequency = freq_val,
                                                       n_peak_width = 1, n_std_as_bin = 1)

In [ ]:
dr_bias, dr = compute_resistance(bias,fine_corrected_data)

# power_offset = 

plot_differential_resistance_map(power, bias, dr,
                                 power_offset = power_offset,
                                 cvmax = 120)

plt.savefig(f'Figs/Shapiro/dR-map_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{Vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')

In [ ]:
I_c =

voltage_bins, histo_counts = bin_power_shapiro_steps(power, bias, fine_corrected_data, 
                                                     frequency = freq_val, step_fraction = 0.01)

plot_shapiro_histogram(power, voltage_bins,
                       histo_counts,savgol_windowl =3,
                       savgol_polyorder = 1,
                       I_c = I_c,
                       transpose = True,
                       mark_steps = [0,-1,-2])

plt.savefig(f'Figs/Shapiro/histo-map_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{Vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')

In [ ]:
voltage_bins, histo_counts = bin_power_shapiro_steps(power, bias, fine_corrected_data, 
                                                     frequency = freq_val, step_fraction = 0.1)

plot_step_weights(power, voltage_bins, histo_counts,
                  steps = [-3,-2,-1,0,1,2,3],
                  savgol_windowl = 13,
                  savgol_polyorder = 1,
                  ic = I_c*1e-6,
                  counts_limits = [0,0.5])

plt.savefig(f'Figs/Shapiro/steps-weights_{SAMPLE_ID}_freq:{freq[22:]}_field-y:{field_y[16:]}_field-z:{field_z[16:]}_Vg:{Vg[4:]}_{FILE_NUM}.pdf', bbox_inches = 'tight')